# Lesson 2: RAG Triad of metrics

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [3]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [4]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./CELEX_32016R0679_EN_TXT.pdf"]
).load_data()

In [5]:
from llama_index import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [6]:
from utils import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [8]:
output = sentence_window_engine.query(
    "How should an organization manage user authorizations to ensure data security under GDPR?")
output.response

'An organization should adopt internal policies and implement measures that meet the principles of data protection by design and data protection by default. This includes minimizing the processing of personal data, pseudonymizing personal data as soon as possible, ensuring transparency regarding the functions and processing of personal data, enabling data subjects to monitor data processing, and creating and improving security features. Additionally, when developing applications, services, and products that involve processing personal data, organizations should consider the right to data protection and ensure that controllers and processors can fulfill their data protection obligations. The principles of data protection by design and by default should also be considered in the context of public tenders.'

## Feedback functions

In [9]:
import nest_asyncio

nest_asyncio.apply()

In [10]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### 1. Answer Relevance

In [11]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [12]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [13]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [14]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 3. Groundedness

In [15]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [19]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [18]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

TypeError: Object of type 'OpenAI' is not JSON serializable

In [20]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [21]:
eval_questions

['What are the essential security measures for effective teleworking under GDPR?',
 'What organizational measures should be taken to ensure compliance with the GDPR in terms of data security?',
 'What technical measures are essential for ensuring the security of personal data under GDPR?',
 'What are the essential technical measures to secure equipment and workstations for protecting personal data?',
 'How should an organization protect its premises to ensure the security of personal data?',
 'How should an organization implement user authentication to protect personal data under GDPR requirements?',
 'How should an organization manage user authorizations to ensure data security under GDPR?',
 'What is pseudonymisation and how should it be implemented under GDPR?',
 'How does encryption and hash functions contribute to GDPR compliance?',
 'What does data anonymisation involve under GDPR, and how is it distinguished from pseudonymisation?']

In [22]:
eval_questions.append("What are the benefits of conducting a DPIA?")

In [23]:
eval_questions

['What are the essential security measures for effective teleworking under GDPR?',
 'What organizational measures should be taken to ensure compliance with the GDPR in terms of data security?',
 'What technical measures are essential for ensuring the security of personal data under GDPR?',
 'What are the essential technical measures to secure equipment and workstations for protecting personal data?',
 'How should an organization protect its premises to ensure the security of personal data?',
 'How should an organization implement user authentication to protect personal data under GDPR requirements?',
 'How should an organization manage user authorizations to ensure data security under GDPR?',
 'What is pseudonymisation and how should it be implemented under GDPR?',
 'How does encryption and hash functions contribute to GDPR compliance?',
 'What does data anonymisation involve under GDPR, and how is it distinguished from pseudonymisation?',
 'What are the benefits of conducting a DPIA?'

In [24]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [25]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_8ec33b90d51e6f3cc8959703f5eefb59,"""What are the essential security measures for ...","""The essential security measures for effective...",-,"{""record_id"": ""record_hash_8ec33b90d51e6f3cc89...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-03T08:17:48.466835"", ""...",2024-06-03T08:17:55.740594,1.0,0.40,2.325000,[{'args': {'prompt': 'What are the essential s...,[{'args': {'question': 'What are the essential...,[{'args': {'source': '1). household activit...,7,752,0.001170
1,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_f81594e2173d2c34f9cb4f123a6bea17,"""What organizational measures should be taken ...","""Organizational measures that should be taken ...",-,"{""record_id"": ""record_hash_f81594e2173d2c34f9c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-03T08:17:55.924461"", ""...",2024-06-03T08:18:03.058293,1.0,0.55,0.250000,[{'args': {'prompt': 'What organizational meas...,[{'args': {'question': 'What organizational me...,[{'args': {'source': 'Risk should be evaluated...,7,964,0.001488
2,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_a338ea74b6ce100f8638e2d7a08c9d83,"""What technical measures are essential for ens...","""Appropriate technical measures essential for ...",-,"{""record_id"": ""record_hash_a338ea74b6ce100f863...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-03T08:18:03.242302"", ""...",2024-06-03T08:18:11.847055,1.0,0.95,1.000000,[{'args': {'prompt': 'What technical measures ...,[{'args': {'question': 'What technical measure...,[{'args': {'source': '4.5.2016 L 119/47 Offici...,8,861,0.001327
3,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_6cbfe28b5318978a93c7757437ba332f,"""What are the essential technical measures to ...","""Essential technical measures to secure equipm...",-,"{""record_id"": ""record_hash_6cbfe28b5318978a93c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-03T08:18:12.042262"", ""...",2024-06-03T08:18:18.546759,1.0,0.75,0.083333,[{'args': {'prompt': 'What are the essential t...,[{'args': {'question': 'What are the essential...,[{'args': {'source': '(78) The prote ction of...,6,870,0.001332
4,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_bc4386738363178202dd902acaa25b3f,"""How should an organization protect its premis...","""An organization should ensure appropriate sec...",-,"{""record_id"": ""record_hash_bc4386738363178202d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-03T08:18:18.737686"", ""...",2024-06-03T08:18:26.045565,1.0,0.95,1.000000,[{'args': {'prompt': 'How should an organizati...,[{'args': {'question': 'How should an organiza...,[{'args': {'source': 'Personal data should be ...,7,1017,0.001542


In [26]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Groundedness,Answer Relevance
0,"""What are the essential security measures for effective teleworking under GDPR?""","""The essential security measures for effective teleworking under GDPR include implementing appropriate technological protection and organizational measures to promptly detect any personal data breaches, informing the supervisory authority and the data subject without undue delay in case of a breach likely to result in a high risk to the rights and freedoms of the individual, and communicating the nature of the breach along with recommendations to mitigate potential adverse effects to the data subject as soon as reasonably feasible.""",0.40,2.325000,1.0
1,"""What organizational measures should be taken to ensure compliance with the GDPR in terms of data security?""","""Organizational measures that should be taken to ensure compliance with the GDPR in terms of data security include adopting internal policies, implementing measures that align with the principles of data protection by design and data protection by default, minimizing the processing of personal data, pseudonymizing personal data at the earliest opportunity, ensuring transparency regarding the functions and processing of personal data, enabling data subjects to monitor data processing, and creating and improving security features.""",0.55,0.250000,1.0
2,"""What technical measures are essential for ensuring the security of personal data under GDPR?""","""Appropriate technical measures essential for ensuring the security of personal data under GDPR include implementing data protection principles such as data minimization, pseudonymization, and ensuring that only necessary personal data for each specific purpose are processed. Additionally, measures should be in place to restrict access to personal data without individual intervention and to evaluate and address risks associated with data processing operations.""",0.95,1.000000,1.0
3,"""What are the essential technical measures to secure equipment and workstations for protecting personal data?""","""Essential technical measures to secure equipment and workstations for protecting personal data include implementing appropriate security features, enabling encryption where necessary, restricting access to authorized personnel only, regularly updating software and systems to address vulnerabilities, and ensuring that only necessary personal data is processed on each workstation.""",0.75,0.083333,1.0
4,"""How should an organization protect its premises to ensure the security of personal data?""","""An organization should ensure appropriate security measures are in place to protect its premises, including preventing unauthorized access to or use of personal data and the equipment used for processing.""",0.95,1.000000,1.0
5,"""How should an organization implement user authentication to protect personal data under GDPR requirements?""","""An organization should implement user authentication measures to protect personal data under GDPR requirements by ensuring that only authorized individuals have access to personal data. This can involve using secure login credentials, such as passwords or biometric data, and implementing multi-factor authentication where possible. Additionally, organizations should regularly review and update their authentication processes to maintain the security of personal data in compliance with GDPR regulations.""",0.35,0.433333,1.0
6,"""How should an organization manage user authorizations to ensure data security under GDPR?""","""An organization should adopt internal policies and implement measures that meet the principles of data protection by design and data protection by default. This includes minimizing the processing of personal data, pseudonymizing personal data, ensuring transparency in data processing, enabling data subjects to monitor data processing, and creating and improving security features. Additionally, when developing applications, servic

In [27]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
App_1,0.554545,0.754545,0.990909,7.272727,0.001898


In [ ]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…